<a href="https://colab.research.google.com/github/sungshin98/4stup/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
g++ is already the newest version (4:9.3.0-1ubuntu2).
python3-dev is already the newest version (3.8.2-0ubuntu2).
python-d

In [21]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [22]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import re
%matplotlib inline
import csv
import gensim
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 스크립트 별로 디렉토리와 파일을 따로 생성

In [23]:
files = [f for f in os.listdir("/content/drive/MyDrive/Session01") if ".wav" not in f]
sorted_files = sorted(files, key=lambda x: int(re.findall(r"script(\d+)", x)[0]))

# 정규식 패턴으로 파일 이름에서 숫자 추출
def get_number_from_filename(filename):
    return int(re.findall(r"script(\d+)", filename)[0])

directory = f"script"

# 파일 이름에서 숫자를 추출하여 디렉토리 생성
def create_directory_for_file(filename):
    number = get_number_from_filename(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

# 파일을 script 뒤의 숫자를 기준으로 분할하여 저장
for file in sorted_files:
    with open("/content/drive/MyDrive/Session01/"+file, "r",encoding="euc-kr") as f:
        text = f.read()
        directory = create_directory_for_file(file)
        with open(f"{directory}/{file}", "w") as f2:
            f2.write(text)

#분석

In [24]:
import os
import gensim
from konlpy.tag import Okt
from gensim.models import Word2Vec
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/ratings_train.txt', sep='\t')
train_df.dropna(inplace=True)

okt=Okt()
sentences=[]

for document in train_df['document']:
    tokens = okt.morphs(document)
    sentences.append(tokens)


texts = train_df['document'].tolist()
sentiments = train_df['label'].tolist()

In [25]:
model = Word2Vec(sentences, vector_size=200, window=5, min_count=5, workers=16, sg=1)

In [28]:
def predict_sentiment(text):
    tokens = okt.morphs(text)
    vectors = []
    for token in tokens:
        if token in model.wv.key_to_index:
            vector = model.wv[token]
            vectors.append(vector)
    if len(vectors) == 0:
        return 'unknown'
    else:
        # calculate the sentence vector by taking the mean of all the word vectors
        sentence_vector = np.mean(vectors, axis=0)
        # calculate cosine similarities
        vector_list = [model.wv[x] for x in texts if x in model.wv.key_to_index]
        sim_scores = cosine_similarity([sentence_vector], vector_list)
        # exclude NaN values from sim_scores
        sim_scores = sim_scores[~np.isnan(sim_scores)]

        if len(sim_scores) == 0:
            return 'unknown'
        else:
            # select the index of the most similar vector
            idx = np.argmax(sim_scores)
            # find the corresponding text for the most similar vector
            result = texts[idx]
            # return the sentiment score for the corresponding text
            print(text)
            print(result)
            return sentiments[texts.index(result)]


dir_path = '/content/script'
file_list = os.listdir(dir_path)

if '.DS_Store' in file_list:
    file_list.remove('.DS_Store')

txt_files = []

for root, dirs, files in os.walk(dir_path):
    for file in files:
        txt_files.append(os.path.join(root, file))

for file_path in txt_files:
    with open(file_path, 'r') as f:
        text = f.read()
        predict_score = predict_sentiment(text)
        if predict_score != 'unknown':
            print(f'{file_path}: {predict_score:.2f}')
        else:
            print(f'{file_path}: {predict_score}')

어 개발사마다 막 구글 쪽도 있고 카카오도 있고 네이버도 있고 막 엄청 다양하다 보니까 b/ 뭐가 좋은 u/ 뭐가 좋은 거 같애 오빠?

최고! 더 무슨평가가 필요한가 ~~탑!
/content/script/Sess01_script05_User001F_014.txt: 1.00
진짜 이름 이 만 그거 하고 거의 밖에서 놀지도 않고 술 먹거나 이런 것도 아니고 그냥 서로 이제 얼굴 이름만 아는 인사만 하는 룸메더라고.

최고! 더 무슨평가가 필요한가 ~~탑!
/content/script/Sess01_script03_User002M_014.txt: 1.00
응 어제 이제 되게 내가 너무 멀어서 혹시 바로 앞에 있는 물을 좀 주실 수 있나요 뭐 이렇게 하는 건 되는데 되게 저기 멀리 있는데 내 나 내가 갈수도 있는데 혹시 저기 물 좀 갖다 주실 수 있나요 이거는 나는 말이 안 된다고 생각해.

최고! 더 무슨평가가 필요한가 ~~탑!
/content/script/Sess01_script02_User002M_022.txt: 1.00
나는 괜찮았다고 생각했어, 그게.

최고! 더 무슨평가가 필요한가 ~~탑!
/content/script/Sess01_script02_User001F_004.txt: 1.00
그래서 같이 어 그러면 같이 방을 볼래요 하면서 같이 방을 보다가 그 결국은 둘이 같이 룸메를 해서 하숙집에 들어갔어.

13구역이라는 머리를 떼고 나왔을면 더 좋았을 것을 암튼 상반기 최고의 액션 영화다
/content/script/Sess01_script03_User002M_009.txt: 1.00
되게 가부장적인 아버지라고 생각하면 b/ 우리 아버지도 좀 그런 거 있는 거 같애.

최고! 더 무슨평가가 필요한가 ~~탑!
/content/script/Sess01_script06_User002M_001.txt: 1.00
c/ 왜냐면 n/ 어떤 요즘 그 간호사 직 같은 경우에는 여성 비율이 상당히 높은 편이잖아.

기적은 이미 일어났다.
/content/script/Sess